# Task 1.2 DataPreparation

In [ ]:
import math
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sn
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
# load the datasets
df_incidents = pd.read_csv('../datasets_clean/incidents_clean.csv', sep=',', index_col=0)
df_poverty = pd.read_csv('../datasets_clean/poverty_clean.csv', sep=',', index_col=0)
df_elections = pd.read_csv('../datasets_clean/elections_clean.csv', sep=',', index_col=0)

In [ ]:
# types of the columns
df_incidents.dtypes


In [ ]:
# fix the type of the columns
df_incidents['date'] = pd.to_datetime(df_incidents['date'])
df_incidents['state'] = df_incidents['state'].astype('string')
df_incidents['city_or_county'] = df_incidents['city_or_county'].astype('string')
df_incidents['city_or_county'] = df_incidents['city_or_county'].astype('string')
df_incidents['incident_characteristics1']=df_incidents['incident_characteristics1'].astype('string')
df_incidents['address'] = df_incidents['address'].astype('string')

## Joining datasets


In [ ]:
# We join the datasets df_incidents and df_poverty on the columns
# -1 create a temporary column year in df_incidents
df_incidents['year']=df_incidents['date'].dt.year

In [ ]:
# -2 join the datasets
df_incidents = pd.merge(df_incidents, df_poverty, on=['state', 'year'], how='left')

In [ ]:
df_incidents.head()

In [ ]:
# We join the datasets df_incidents and df_elections on the columns
# 1. turn congressional_district =0 in 1
df_elections['congressional_district'] = df_elections['congressional_district'].replace(0, 1)

In [ ]:
df_elections['state'] = df_elections['state'].astype('string')

In [ ]:
df_elections['year'] = df_elections['year'].astype('float64')
df_elections['congressional_district']=df_elections['congressional_district'].astype('float64')
df_elections['state']=df_elections['state'].astype('string')

df

In [ ]:
# join
df_incidents = pd.merge(df_incidents, df_elections, on=['state', 'congressional_district', 'year'], how='left')
df_incidents.head()

# Datasets

In [ ]:
df_incidents.head()

In [ ]:
df_poverty.head()

In [ ]:
df_elections.head()

# Definition of new indicators

## Severity of the incident
We start considering the ratio of killed, injured, and unharmed participants in the incidents with respect to the total number of participants. Subsequently, we define the _severity_ of the incidents as the weighted mean of the ratios just defined. We assign to `ratio_killed` the weight 0.5, to `ratio_injured` weight 0.3 and to `ratio_unharmed` weight 0.2.

In [ ]:
total = df_incidents['n_killed'] + df_incidents['n_injured'] + df_incidents['n_unharmed']
df_incidents['ratio_killed'] = df_incidents['n_killed']/total*100
df_incidents['ratio_injured'] = df_incidents['n_injured']/total*100
df_incidents['ratio_unharmed'] = df_incidents['n_unharmed']/total*100

In [ ]:
df_incidents['severity'] = 0.6*df_incidents['ratio_killed'] + 0.3*df_incidents['ratio_injured'] + 0.1*df_incidents['ratio_unharmed']

In [ ]:
df_incidents['severity'].describe()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df_incidents['severity'], bins=10)

In [ ]:
# We drop the columns of the ratios
df_incidents.drop(['ratio_killed', 'ratio_injured', 'ratio_unharmed'], axis=1)

## Injured and killed in a given place and time
We investigate how many injured and killed people have been involved w.r.t the total injured and
killed people in the same congressional district in a month

In [ ]:
df_incidents.date=pd.to_datetime(df_incidents.date)

In [ ]:
#- raggruppare in base al mese, congressional district, stato
#- injured / tot. injured , killed / tot. killed in congr. and month

df_incidents['year_month'] = df_incidents.date.dt.strftime('%y-%m')
df_incidents.year_month.isnull().sum()

tot_killed = df_incidents.groupby(['state', 'congressional_district', 'year_month'])['n_killed'].sum()
# Convert the Series to a DataFrame and reset the index
tot_killed_df = tot_killed.reset_index()

# Rename the 'n_killed' column to a desired name
tot_killed_df = tot_killed_df.rename(columns={'n_killed': 'total_killed'})

# Print the resulting DataFrame
print(tot_killed_df)


#Same for the injured
tot_injured = df_incidents.groupby(['state', 'congressional_district', 'year_month'])['n_injured'].sum()
# Convert the Series to a DataFrame and reset the index
tot_injured_df = tot_injured.reset_index()

# Rename the 'n_killed' column to a desired name
tot_injured_df = tot_injured_df.rename(columns={'n_injured': 'total_injured'})

In [ ]:
# TODO: We recall that in the elections dataset, for the states that have just one congressional district, the number of the district was 0. We replace the zeros with 1

In [ ]:
# We join the 'total_killed' and 'total_injured' columns to the incidents dataset
df_incidents =pd.merge( df_incidents, tot_killed_df, on=['state', 'congressional_district', 'year_month'])

In [ ]:
df_incidents = pd.merge(df_incidents, tot_injured_df, on=['state', 'congressional_district', 'year_month'])
df_incidents.head()

In [ ]:
# We distinguish between the case with total_killed = 0, and total_killed != 0

def f(x, y):
    if y!=0:
        return x/y
    else:
        return 0
df_incidents['killed_by_district'] = df_incidents.apply(lambda row: f(row['n_killed'], row['total_killed']), axis=1)

df_incidents['injured_by_district'] = df_incidents.apply(lambda row: f(row['n_injured'], row['total_injured']), axis=1)

In [ ]:
# Let's see the distribution of the feature 'killed_by_district'
plt.figure(figsize=(10, 5))
plt.hist(df_incidents['killed_by_district'], bins=5)

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df_incidents['injured_by_district'], bins=5)

# Aggregated dataset

We group the incidents that happened in the same congressional district of each state: label [state, congressional district] (and eventually in the same year, month).

We consider the numerical features of the dataset df_incidents:
- the mean of 'avg_age_participants'
- the sum of the features 'n_participants_child'-'n_participants'

We also add the 'povertyPercentage' feature of df_poverty for each state, year.

Finally, given the year of the incident, we add the 'party', 'candidatevotes', 'totalvotes' features from the dataset df_elections for each [state, congressional district].